### Sprint 2 - Decision Tree

Asuntos pendientes:
- Crear el *Decision tree*.
- Meter "más gráficas" en el EDA.
- Crear modelos separados para "casual" y "registered".


[ ] Separar los dfs en "casual" y "registered".

[ ] Hemos limpiado pepe.

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

/home/sanfermen/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Empezamos con los casual estandarizados

In [18]:
df_casual = pd.read_csv("../datos/bikes_casual_estan.csv", index_col= 0)
df_casual.head(2)

,instant,yr,holiday,workingday,atemp,hum,windspeed,casual,weekday,weathersit_map,season_map
0,1,0,1,0,-0.456399,0.855582,-0.208909,331,1,1,0
1,2,0,0,1,-0.492863,0.332722,0.687550,131,4,1,0


In [19]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X = df_casual.drop("casual", axis = 1)
y = df_casual["casual"]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [21]:
# creamos el objeto del modelo, al igual que hacíamos en la regresión lineal
arbol = DecisionTreeRegressor(random_state =0)

# ajustamos el modelo, igual que en la regresión lienal. 
arbol.fit(x_train, y_train)


DecisionTreeRegressor(random_state=0)

In [22]:
# max features. Como vemos, debemos poner en nuestro modelo una profudidad máxima de 4. 

max_features = np.sqrt(len(x_train.columns))
max_features

3.1622776601683795

In [23]:
# max depth
print(arbol.tree_.max_depth)

22


In [24]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [25]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [26]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decission Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,372.938356,349259.472603,590.981787,0.273018,test,Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree I


In [27]:
df_casual.shape

(730, 11)

Hacemos GridSearchCV

In [28]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [12,14, 16], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [29]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [30]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [12, 14, 16], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [31]:
# este método nos esta diciendo que el mejor modelo es aquel que tiene una profundidad de 6, que usa 4 variables predictoras para construir el modelo y que tiene  un min_samples_leaf y un min_samples_split de 10. 
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=14, max_features=2, min_samples_leaf=10,
                      min_samples_split=10)

In [32]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [33]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2


,MAE,MSE,RMSE,R2,set,modelo
0,351.317998,238609.440289,488.476653,0.503336,test,Decision tree II
1,338.164445,223894.050467,473.174440,0.521293,train,Decision tree II


Registered

In [34]:
df_registered = pd.read_csv("../datos/bikes_registered_estan.csv", index_col= 0)
df_registered.head(2)

,instant,yr,holiday,workingday,atemp,hum,windspeed,registered,weekday,weathersit_map,season_map
0,1,0,1,0,-0.456399,0.855582,-0.208909,654,1,1,0
1,2,0,0,1,-0.492863,0.332722,0.687550,670,4,1,0


In [35]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X2 = df_registered.drop("registered", axis = 1)
y2 = df_registered["registered"]

In [36]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.2, random_state = 42)

In [37]:
# creamos el objeto del modelo, al igual que hacíamos en la regresión lineal
arbol2 = DecisionTreeRegressor(random_state =0)

# ajustamos el modelo, igual que en la regresión lienal. 
arbol2.fit(x_train2, y_train2)

DecisionTreeRegressor(random_state=0)

In [38]:

max_features2 = np.sqrt(len(x_train2.columns))
max_features2

3.1622776601683795

In [39]:
print(arbol2.tree_.max_depth)

20


In [40]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_dt2 = arbol2.predict(x_test2)
y_pred_train_dt2 = arbol2.predict(x_train2)

In [42]:

dt_results2 = metricas(y_test2, y_train2, y_pred_test_dt2, y_pred_train_dt2, "Decission Tree II")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,566.089041,629413.061644,793.355571,0.708412,test,Decission Tree II
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree II


In [43]:
param2 = {"max_depth": [10, 12,14, 16, 18], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [44]:
gs2 = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [45]:
gs2.fit(x_train2, y_train2)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 12, 14, 16, 18],
                         'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [46]:
mejor_modelo2 = gs2.best_estimator_
mejor_modelo2

DecisionTreeRegressor(max_depth=12, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [48]:
y_pred_test_dt3 = mejor_modelo2.predict(x_test2)
y_pred_train_dt3 = mejor_modelo2.predict(x_train2)

In [49]:
dt_results3 = metricas(y_test2, y_train2, y_pred_test_dt3, y_pred_train_dt3, "Decision tree III")
dt_results3

,MAE,MSE,RMSE,R2,set,modelo
0,594.656826,576276.487237,759.128769,0.733029,test,Decision tree III
1,543.159644,576369.456574,759.190000,0.768812,train,Decision tree III
